## Decision Tree
Name: Yang Chen <br>

Result requirement as following:
1. Most common label in the training data
2. Entropy of the training data
3. best feature and its information gain in training data
4. accuracy on the training set
5. accuracy on the test set
6. average accuracies from the cross-validation for each depth
7. best depth
8. accuracy on the test set using the best depth

### Implementing the full tree with information entropy

In [1]:
# First, we write the file reader to read the data in
import numpy as np
import pandas as pd
from pprint import pprint
'''
read the data from the corresponding csv file
input: csv file
output: features dataframe, label dataframe
'''

def fileReader(file, features):
    # in total we have 123 features
    df = pd.read_csv(file, header=None, index_col=False)
    label = df[0]
    df.drop(columns=0, inplace=True)
    df.columns = features
    return df, label

# load the train, test, fold data
features = [str(i) for i in range(1, 124, 1)]
train_feature, train_label = fileReader("./data_hw1_wCSV/a1a_train.csv", features)
test_feature, test_label = fileReader("./data_hw1_wCSV/a1a_test.csv", features)
fold1_feature, fold1_label = fileReader('./data_hw1_wCSV/CVfolds/fold1.csv', features)
fold2_feature, fold2_label = fileReader('./data_hw1_wCSV/CVfolds/fold2.csv', features)
fold3_feature, fold3_label = fileReader('./data_hw1_wCSV/CVfolds/fold3.csv', features)
fold4_feature, fold4_label = fileReader('./data_hw1_wCSV/CVfolds/fold4.csv', features)
fold5_feature, fold5_label = fileReader('./data_hw1_wCSV/CVfolds/fold5.csv', features)

# check the shape of each data
print(train_feature.shape)
print(test_feature.shape)
print(fold1_feature.shape)
print(fold2_feature.shape)
print(fold3_feature.shape)
print(fold4_feature.shape)
print(fold5_feature.shape)

# visualize the head of the train feature set
print(train_feature.head())
print(train_label.head())
print(type(train_label))

(1558, 123)
(30956, 123)
(312, 123)
(312, 123)
(312, 123)
(312, 123)
(310, 123)
   1  2  3  4  5  6  7  8  9  10  ...  114  115  116  117  118  119  120  121  \
0  0  0  1  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0    0   
1  0  0  1  0  0  1  0  0  0   0  ...    0    0    0    0    0    0    0    0   
2  0  0  0  1  0  1  0  0  0   0  ...    0    0    0    0    0    0    0    0   
3  0  0  0  0  1  1  0  0  0   0  ...    0    0    0    0    0    0    0    0   
4  0  1  0  0  0  1  0  0  0   0  ...    0    0    0    0    0    0    0    0   

   122  123  
0    0    0  
1    0    0  
2    0    0  
3    0    0  
4    0    0  

[5 rows x 123 columns]
0   -1
1   -1
2   -1
3   -1
4   -1
Name: 0, dtype: int64
<class 'pandas.core.series.Series'>


In [51]:
# todo: define the node class to store the following
"""
1. feature
2. left / right node
3. label (class)
if this is a feature node then label is None;
if this is a label node then feature is None;
Because, for this particular dataset, for each feature, it is binary; therefore, we just use
the binary tree. Of course, for more than two values, we need to modify it.
"""
class Node:
    # we define the node data structure to store feature, left node, right node, label
    # if it is a feature node, then label should be None
    # if it is a label node, the feature should be None
    def __init__(self):
        self.feature = None # choose from the feature list [1, 2, ... , 123]
        # self.left = None
        # self.right = None
        # generalize this for more values for later use
        # because, there is possibility that we have more that binary values
        self.children = {}
        self.label = None # can only from [-1, 1] from the label set

# todo: define the ID3 algorithm
"""
1. deal with the boundary conditions:
    (a) all belong to one class then make that node as the label node
    (b) if there is no target feature left, we find the most occurred class as the label
2. calculate information gain for each feature and find the biggest one (need support functions)
{entropy, information gain, most_important-feature}
3. recursively build the tree (how to create is a very important step!!)
4. return the tree.
"""
"""
Based on the tree node we defined, we use a top-bottom method to write ID3 algorithm
input: feature dataset, label dataset, features set
notice: this three features will be updated when recursively called
output: tree node
"""
def is_unique(label_data):
    """
    :param label_data:
    :return: boolean whether the label dataset has the same value
    """
    label_array = label_data.to_numpy()
    return (label_array[0] == label_array).all()

def entropy(label_data):
    """
    :param label_data:
    :return: information entropy of corresponding labels under certain condition
    """
    labels = label_data.value_counts().to_list()
    entropy = np.sum([(-labels[i]/len(label_data)*np.log2(labels[i]/len(label_data))) for i in range(len(labels))])
    return entropy

def InfoGain(feature_data, label_data, feature, total_entropy):
    # find out what attribute can this feature take
    # in this case, values can only be 0 or 1
    # but later for generalization, values can take more than two values
    values = feature_data[feature].value_counts().index.to_list()
    weighted_entropy = np.sum([len(feature_data[feature_data[feature] == value])/len(feature_data) * entropy(label_data[feature_data[feature] == value]) for value in values])
    return total_entropy - weighted_entropy

def most_common(label_data):
    return label_data.value_counts().idxmax()


def ID3(feature_data, label_data, features, default):

    # the most commom feature from the previous dataset
    # if there is no data but still features
    if len(label_data) == 0:
        tree_node = Node()
        tree_node.label = default
        return tree_node
    # first deal with the boundary conditions
    # boundary condition 1: if the label has the same value
    if is_unique(label_data):
        # we assign the label to the tree node as a label node and return
        tree_node = Node()
        tree_node.label = label_data.value_counts().idxmax()
        return tree_node

    # boundary condition 2: if the features set is empty
    # if features is empty, it will return False in the if statement
    if not features: # True if the feature list is empty
        # find the most common label in and return as label node
        most_common_label = label_data.value_counts().idxmax()
        tree_node = Node()
        tree_node.label = most_common_label
        return tree_node
    else: # the normal case when the features list is not empty
        # find the feature for the current tree node
        # the information entropy of the current data set
        total_entropy = entropy(label_data)
        # we find the best feature for this tree node
        info_gains = [InfoGain(feature_data, label_data, feature, total_entropy) for feature in features]
        best_feature_index = np.argmax(info_gains)
        print("the largest info gain is %f " % np.max(info_gains))
        best_feature = features[best_feature_index]
        tree_node = Node()
        # create a feature tree node
        tree_node.feature = best_feature
        tree_node.label = None

        # update the data set and feature set
        features = [i for i in features if i != best_feature]
        for value in feature_data[best_feature].value_counts().index.to_list():
            # print(value)
            new_feature_data = feature_data[feature_data[best_feature] == value]
            new_feature_data.drop(columns=best_feature)
            new_label_data = label_data[feature_data[best_feature] == value]
            # if best_feature == "40":
            #     print(tree_node.feature, value, new_label_data.value_counts())
            #     print(features)
            sub_tree = ID3(new_feature_data, new_label_data, features, most_common(feature_data))
            # put the corresponding reuturn tree into the node
            tree_node.children[value] = sub_tree

        return tree_node

In [52]:
features = [str(i) for i in range(1,124,1)]
tree_train = ID3(train_feature, train_label, features, most_common(train_label))

the largest info gain is 0.153945 
the largest info gain is 0.048882 
the largest info gain is 0.019501 
the largest info gain is 0.014309 
the largest info gain is 0.013080 
the largest info gain is 0.012552 
the largest info gain is 0.026052 
the largest info gain is 0.011641 
the largest info gain is 0.026788 
the largest info gain is 0.543564 
the largest info gain is 0.052144 
the largest info gain is 0.092980 
the largest info gain is 0.811278 
the largest info gain is 0.918296 
the largest info gain is 0.469565 
the largest info gain is 0.918296 
the largest info gain is 0.152478 
the largest info gain is 0.233550 
the largest info gain is 0.419973 
the largest info gain is 0.918296 
the largest info gain is 0.059075 
the largest info gain is 0.083048 
the largest info gain is 0.084350 
the largest info gain is 0.063720 
the largest info gain is 0.108849 
the largest info gain is 0.137925 
the largest info gain is 0.198117 
the largest info gain is 0.251629 
the largest info gai

In [53]:
#` we transvers the tree
tree_dict = {}
def traverseTree(root, level = 0):
        """
        traverse function will print all the node in the tree.
        """
        if root:
            if level in tree_dict.keys():
                tree_dict[level].append([root.feature, root.label])
            else:
                tree_dict[level] = [[root.feature, root.label]]
            for value in root.children.values():
                traverseTree(value, level+1)

traverseTree(tree_train)
pprint(tree_dict)

{0: [['40', None]],
 1: [['39', None], ['39', None]],
 2: [['74', None], ['74', None], ['35', None], ['74', None]],
 3: [['51', None],
     ['6', None],
     ['82', None],
     ['11', None],
     ['4', None],
     ['74', None],
     ['76', None],
     ['78', None]],
 4: [['27', None],
     ['67', None],
     ['81', None],
     ['49', None],
     ['32', None],
     ['8', None],
     ['5', None],
     [None, -1],
     ['51', None],
     ['9', None],
     ['10', None],
     ['21', None],
     ['19', None],
     [None, 1],
     [None, 1],
     ['1', None]],
 5: [['42', None],
     ['7', None],
     ['14', None],
     [None, -1],
     [None, -1],
     ['14', None],
     [None, 1],
     [None, -1],
     ['6', None],
     ['2', None],
     ['29', None],
     [None, -1],
     ['10', None],
     [None, 1],
     ['1', None],
     ['74', None],
     ['57', None],
     [None, 1],
     ['91', None],
     ['33', None],
     ['17', None],
     ['48', None],
     ['2', None],
     ['16', None],
     [

In [54]:
# To see how does the tree perform
def accuracy(trained_tree, test_features, test_label):
    """
    Parameters:
        trained_tree: A train tree
        test_features: the dataset of test feature
        test_label: the dataset of test label
    Returns:
        accuracy
    """
    # convert the dataframe to ndarray
    test_features = test_features.to_numpy()
    test_label = test_label.to_numpy()
    N = test_features.shape[0]
    print(N)

    # Set the counter to 0
    num_of_correct_predictions = 0

    for i in range(N):
        if predict(trained_tree, test_features[i]) == test_label[i]:
            num_of_correct_predictions += 1

    return num_of_correct_predictions / N


def predict(node, test_instance):
    """
    Parameters:
        node: A trained tree node
        test_instance: A single test instance
    Returns:
        predicted label
    """
    # Boundary Condition
    if len(node.children) == 0:
        return node.label
    else:
        # get corresponding value
        # value = test_instance[int(node.feature)]
        value = test_instance[int(node.feature) - 1]

        # Follow the tree to get the corresponding label of the test instance
        if value in node.children.keys():
            # Recursive call
            return predict(node.children[value], test_instance)

In [55]:
train_acc = accuracy(tree_train, train_feature, train_label)
print(1.0 - train_acc)
test_acc = accuracy(tree_train, test_feature, test_label)
print(1.0 - test_acc)

1558
0.0
30956
0.22615971055691952


### Limiting the tree depth and cross-validation to find the best tree depth (hyperparameter)

In [56]:
# we include a hyperparameter depth (level) to to record and indicate the level of the tree
# Based on the tree level, we terminate and output the root node of the tree
# The structure is almost the same but with a new boundary condition

def ID3_pruned(feature_data, label_data, features, default, depth, level = 0):

    # the most commom feature from the previous dataset
    # if there is no data but still features
    if len(label_data) == 0:
        tree_node = Node()
        tree_node.label = default
        return tree_node
    # first deal with the boundary conditions
    # boundary condition 1: if the label has the same value
    if is_unique(label_data):
        # we assign the label to the tree node as a label node and return
        tree_node = Node()
        tree_node.label = label_data.value_counts().idxmax()
        return tree_node

    # boundary condition 2: if the features set is empty
    # or the tree level equals to the hyperparameter depth
    # if features is empty, it will return False in the if statement
    if (not features) or (level == depth): # True if the feature list is empty
        # find the most common label in and return as label node
        most_common_label = label_data.value_counts().idxmax()
        tree_node = Node()
        tree_node.label = most_common_label
        return tree_node
    else: # the normal case when the features list is not empty
        # find the feature for the current tree node
        # the information entropy of the current data set
        total_entropy = entropy(label_data)
        # we find the best feature for this tree node
        info_gains = [InfoGain(feature_data, label_data, feature, total_entropy) for feature in features]
        best_feature_index = np.argmax(info_gains)
        # print("the largest info gain is %f " % np.max(info_gains))
        best_feature = features[best_feature_index]
        tree_node = Node()
        # create a feature tree node
        tree_node.feature = best_feature
        tree_node.label = None

        # update the data set and feature set
        features = [i for i in features if i != best_feature]
        for value in feature_data[best_feature].value_counts().index.to_list():
            # print(value)
            new_feature_data = feature_data[feature_data[best_feature] == value]
            new_feature_data.drop(columns=best_feature)
            new_label_data = label_data[feature_data[best_feature] == value]
            # if best_feature == "40":
            #    print(tree_node.feature, value, new_label_data.value_counts())
            #    print(features)
            sub_tree = ID3_pruned(new_feature_data, new_label_data, features, most_common(feature_data), depth, level+1)
            # put the corresponding reuturn tree into the node
            tree_node.children[value] = sub_tree

        return tree_node

### Cross-Validation to choose the best tree depth with good capacity to generalize

In [34]:
# we iterate through each depth and each fold as the validation set
depths = [1, 2, 3, 4, 5]
train_feature_folds = [fold1_feature, fold2_feature, fold3_feature, fold4_feature, fold5_feature]
train_label_folds = [fold1_label, fold2_label, fold3_label, fold4_label, fold5_label]
num_of_folds = len(train_feature_folds)
accs = np.zeros([len(depths), num_of_folds])

for depth in depths:
    for i in range(num_of_folds):
        print("We are doing the cross_validation on depth: %d and the %d group as validation set!" % (depth, i))
        train_concat_features = pd.concat([train_feature_folds[j] for j in range(num_of_folds) if j != i], ignore_index=True)
        train_concat_labels = pd.concat([train_label_folds[j] for j in range(num_of_folds) if j != i], ignore_index=True)
        tree = ID3_pruned(train_concat_features, train_concat_labels, features, most_common(train_concat_labels), depth=depth)
        acc = accuracy(tree, train_feature_folds[i], train_label_folds[i])
        accs[depth-1][i] = acc

print("Finished!")


We are doing the cross_validation on depth: 1 and the 0 group as validation set!
312
We are doing the cross_validation on depth: 1 and the 1 group as validation set!
312
We are doing the cross_validation on depth: 1 and the 2 group as validation set!
312
We are doing the cross_validation on depth: 1 and the 3 group as validation set!
312
We are doing the cross_validation on depth: 1 and the 4 group as validation set!
310
We are doing the cross_validation on depth: 2 and the 0 group as validation set!
312
We are doing the cross_validation on depth: 2 and the 1 group as validation set!
312
We are doing the cross_validation on depth: 2 and the 2 group as validation set!
312
We are doing the cross_validation on depth: 2 and the 3 group as validation set!
312
We are doing the cross_validation on depth: 2 and the 4 group as validation set!
310
We are doing the cross_validation on depth: 3 and the 0 group as validation set!
312
We are doing the cross_validation on depth: 3 and the 1 group as 

In [37]:
# analyze the cross-validation results

avg_accs = np.mean(accs, axis=1)
stds = np.std(accs, axis=1)
for i in range(len(depths)):
    print("The average cross-validated accuracy with depth %d is %f" % (i+1, avg_accs[i]))
    print("The average cross-validated std with depth %d is %f" % (i+1, stds[i]))

The average cross-validated accuracy with depth 1 is 0.755459
The average cross-validated std with depth 1 is 0.011271
The average cross-validated accuracy with depth 2 is 0.807465
The average cross-validated std with depth 2 is 0.019260
The average cross-validated accuracy with depth 3 is 0.810041
The average cross-validated std with depth 3 is 0.020576
The average cross-validated accuracy with depth 4 is 0.804913
The average cross-validated std with depth 4 is 0.018073
The average cross-validated accuracy with depth 5 is 0.805554
The average cross-validated std with depth 5 is 0.016809


In [39]:
# From the cross-validation, we choose the depth 3 as the hyperparameter to train on the whole dataset

full_tree = ID3_pruned(train_feature, train_label, features, most_common(train_label), depth=3)
train_acc_full = accuracy(full_tree, train_feature, train_label)
print("The train error on whole training set is %f" % (1-train_acc_full))
test_acc_full = accuracy(full_tree, test_feature, test_label)
print("The test error on whole test set is %f" % (1-test_acc_full))

1558
The train error on whole training set is 0.183569
30956
The test error on whole test set is 0.177801


### Using Gini Index as the measure of impunity to train the tree
$$\sum_ip_i(1-p_i)$$   <br>
where i is each label

In [45]:
# Basically, the whole idea of the algorithm does not change. We only need to update measure of impunity
# from information entropy to gini index.

def gini(label_data):
    """
    :param label_data:
    :return: gini index of this label_data
    """
    labels = label_data.value_counts().to_list() # how many different values
    gini = np.sum([(labels[i]/len(label_data)*(1 - (labels[i]/len(label_data)))) for i in range(len(labels))])
    return gini

# update the info gain method
def InfoGain_gini(feature_data, label_data, feature, total_gini):
    # find out what attribute can this feature take
    # in this case, values can only be 0 or 1
    # but later for generalization, values can take more than two values
    values = feature_data[feature].value_counts().index.to_list()
    weighted_gini = np.sum([len(feature_data[feature_data[feature] == value])/len(feature_data) * gini(label_data[feature_data[feature] == value]) for value in values])
    return total_gini - weighted_gini

# update the ID3 algorithm with gini
# actually we can add a new parameter to indicate with measure we will use to calculate the impurity
def ID3_gini(feature_data, label_data, features, default):

    # the most commom feature from the previous dataset
    # if there is no data but still features
    if len(label_data) == 0:
        tree_node = Node()
        tree_node.label = default
        return tree_node
    # first deal with the boundary conditions
    # boundary condition 1: if the label has the same value
    if is_unique(label_data):
        # we assign the label to the tree node as a label node and return
        tree_node = Node()
        tree_node.label = label_data.value_counts().idxmax()
        return tree_node

    # boundary condition 2: if the features set is empty
    # if features is empty, it will return False in the if statement
    if not features: # True if the feature list is empty
        # find the most common label in and return as label node
        most_common_label = label_data.value_counts().idxmax()
        tree_node = Node()
        tree_node.label = most_common_label
        return tree_node
    else: # the normal case when the features list is not empty
        # find the feature for the current tree node
        # the information entropy of the current data set
        total_measure = gini(label_data)
        # we find the best feature for this tree node
        info_gains = [InfoGain_gini(feature_data, label_data, feature, total_measure) for feature in features]
        best_feature_index = np.argmax(info_gains)
        print("the largest info gain is %f " % np.max(info_gains)) # print out the larget info gain each step
        best_feature = features[best_feature_index]
        tree_node = Node()
        # create a feature tree node
        tree_node.feature = best_feature
        tree_node.label = None

        # update the data set and feature set
        features = [i for i in features if i != best_feature]
        for value in feature_data[best_feature].value_counts().index.to_list():
            # print(value)
            new_feature_data = feature_data[feature_data[best_feature] == value]
            new_feature_data.drop(columns=best_feature)
            new_label_data = label_data[feature_data[best_feature] == value]
            # if best_feature == "40":
            #     print(tree_node.feature, value, new_label_data.value_counts())
            #     print(features)
            sub_tree = ID3_gini(new_feature_data, new_label_data, features, most_common(feature_data))
            # put the corresponding reuturn tree into the node
            tree_node.children[value] = sub_tree

        return tree_node

In [46]:
tree_gini = ID3_gini(train_feature, train_label, features, most_common(train_label))
# show the tree
tree_dict = {}
traverseTree(tree_gini)
pprint(tree_dict)
train_gini_acc = accuracy(tree_gini, train_feature, train_label)
print("The train error on whole training set is %f" % (1-train_gini_acc))
test_gini_acc = accuracy(tree_gini, test_feature, test_label)
print("The test error on whole testing set is %f" % (1-test_gini_acc))

the largest info gain is 0.073741 
0
the largest info gain is 0.011788 
1
the largest info gain is 0.006795 
0
the largest info gain is 0.004694 
0
the largest info gain is 0.004175 
0
the largest info gain is 0.002709 
0
the largest info gain is 0.003168 
0
the largest info gain is 0.001023 
0
the largest info gain is 0.000901 
1
the largest info gain is 0.000154 
0
the largest info gain is 0.000061 
0
1
the largest info gain is 0.004679 
0
1
the largest info gain is 0.244898 
0
1
1
the largest info gain is 0.045000 
0
1
the largest info gain is 0.500000 
1
0
0
the largest info gain is 0.057415 
0
the largest info gain is 0.037415 
0
1
the largest info gain is 0.444444 
0
1
1
1
the largest info gain is 0.444444 
0
1
1
1
the largest info gain is 0.030181 
1
the largest info gain is 0.017690 
0
the largest info gain is 0.018257 
0
the largest info gain is 0.012698 
0
the largest info gain is 0.004057 
0
the largest info gain is 0.005559 
0
the largest info gain is 0.004244 
0
1
the larg